In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data = [
    ("India", "Karnataka", "Bangalore", 100),
    ("India", "Karnataka", "Mysore", 80),
    ("India", "Maharashtra", "Mumbai", 150),
    ("USA", "California", "LA", 200),
]

columns = ["country", "state", "city", "sales"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = spark.createDataFrame(data, columns)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+---------+-----+
|country|      state|     city|sales|
+-------+-----------+---------+-----+
|  India|  Karnataka|Bangalore|  100|
|  India|  Karnataka|   Mysore|   80|
|  India|Maharashtra|   Mumbai|  150|
|    USA| California|       LA|  200|
+-------+-----------+---------+-----+

## Rollup

In [10]:
df.rollup("country", "state", "city").agg(sum("sales").alias("total_sales")).orderBy(col("country").desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+---------+-----------+
|country|      state|     city|total_sales|
+-------+-----------+---------+-----------+
|    USA| California|       LA|        200|
|    USA|       null|     null|        200|
|    USA| California|     null|        200|
|  India|Maharashtra|   Mumbai|        150|
|  India|  Karnataka|   Mysore|         80|
|  India|  Karnataka|     null|        180|
|  India|       null|     null|        330|
|  India|Maharashtra|     null|        150|
|  India|  Karnataka|Bangalore|        100|
|   null|       null|     null|        530|
+-------+-----------+---------+-----------+

In [12]:
df.rollup("state", "city").agg(sum("sales").alias("total_sales")).orderBy(col("state").desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+-----------+
|      state|     city|total_sales|
+-----------+---------+-----------+
|Maharashtra|   Mumbai|        150|
|Maharashtra|     null|        150|
|  Karnataka|Bangalore|        100|
|  Karnataka|   Mysore|         80|
|  Karnataka|     null|        180|
| California|       LA|        200|
| California|     null|        200|
|       null|     null|        530|
+-----------+---------+-----------+

## Cube

In [14]:
df.cube("country", "state", "city").agg(sum("sales").alias("total_sales")).orderBy("country", "state", "city").show(40)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+---------+-----------+
|country|      state|     city|total_sales|
+-------+-----------+---------+-----------+
|   null|       null|     null|        530|
|   null|       null|Bangalore|        100|
|   null|       null|       LA|        200|
|   null|       null|   Mumbai|        150|
|   null|       null|   Mysore|         80|
|   null| California|     null|        200|
|   null| California|       LA|        200|
|   null|  Karnataka|     null|        180|
|   null|  Karnataka|Bangalore|        100|
|   null|  Karnataka|   Mysore|         80|
|   null|Maharashtra|     null|        150|
|   null|Maharashtra|   Mumbai|        150|
|  India|       null|     null|        330|
|  India|       null|Bangalore|        100|
|  India|       null|   Mumbai|        150|
|  India|       null|   Mysore|         80|
|  India|  Karnataka|     null|        180|
|  India|  Karnataka|Bangalore|        100|
|  India|  Karnataka|   Mysore|         80|
|  India|Maharashtra|     null| 

In [15]:
df.filter(col("country") == "USA").cube("country", "state", "city").agg(sum("sales").alias("total_sales")).orderBy("country", "state", "city").show(40)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----+-----------+
|country|     state|city|total_sales|
+-------+----------+----+-----------+
|   null|      null|null|        200|
|   null|      null|  LA|        200|
|   null|California|null|        200|
|   null|California|  LA|        200|
|    USA|      null|null|        200|
|    USA|      null|  LA|        200|
|    USA|California|null|        200|
|    USA|California|  LA|        200|
+-------+----------+----+-----------+

## GroupSets

In [17]:
df.createOrReplaceTempView("sales")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
result = spark.sql("""
SELECT
    country,
    state,
    SUM(sales) AS total_sales
FROM sales
GROUP BY GROUPING SETS (
    (country, state),
    (country),
    ()
)
""")

result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+-----------+
|country|      state|total_sales|
+-------+-----------+-----------+
|  India|       null|        330|
|  India|Maharashtra|        150|
|  India|  Karnataka|        180|
|    USA| California|        200|
|   null|       null|        530|
|    USA|       null|        200|
+-------+-----------+-----------+

In [25]:
result = spark.sql("""
SELECT
    country,
    state,
    city,
    SUM(sales) AS total_sales
FROM sales
GROUP BY GROUPING SETS (
    (country, state, city),
    (country, state),
    (country),
    ()
)
""")

result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+---------+-----------+
|country|      state|     city|total_sales|
+-------+-----------+---------+-----------+
|  India|Maharashtra|   Mumbai|        150|
|  India|       null|     null|        330|
|  India|  Karnataka|   Mysore|         80|
|  India|Maharashtra|     null|        150|
|    USA| California|     null|        200|
|   null|       null|     null|        530|
|    USA|       null|     null|        200|
|  India|  Karnataka|     null|        180|
|    USA| California|       LA|        200|
|  India|  Karnataka|Bangalore|        100|
+-------+-----------+---------+-----------+